## Arrays of objects

A very powerful feature is that we can save instances of classes in an array. This gives us great functionality if we want to save objects for later retrieval using the **numpy.save()** and **numpy.load()** functions.

Python allows object serialisation using cPickle, but serialising with numpy is easier, since we don't have to create file objects beforehand. Performance can be similar, although numpy is slightly faster.

In [1]:
# Import libs and mods
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Observe the following "Account" class
class Cuenta(object):   
    def __init__(self, nombre, saldo_inicial):
        self._nombre = nombre
        self._saldo = saldo_inicial
        self._operaciones = []
    
    def ingresar(self, cantidad):
        self._saldo += cantidad
        self._operaciones.append("Ingreso de " + str(cantidad)+"\nSaldo en cuenta de " + str(self._saldo))
    
    def retirar(self, cantidad):
        if cantidad > self._saldo:
            print ("Operación denegada por la entidad")
            print ("Saldo insuficiente")
            self._operaciones.append("Operación de retirada cancelada, saldo insuficiente")
        else:
            self._operaciones.append("_"*25 + "\nRetirada de " + str(cantidad)+ "\nSaldo en cuenta de " + str(self._saldo))
            self._saldo -= cantidad
        
    def consulta_saldo(self):
        print ("Tu saldo en cuenta es de: " + str(self._saldo))
        
    def consulta_operaciones(self, ultimas = 10):
        if len(self._operaciones)<ultimas:
            ultimas = len(self._operaciones)
        for n in range(ultimas -1, -1, -1):
            print (self._operaciones[n])

In [21]:
# Create 50 random accounts with an initial balance of 2500 euros. 
# For each of them, perform 25 random operations of withdrawal or deposit of amounts from 0 to 200 euros.
# Create random names for the account holders 
# Use the files "data/50_nombres.txt" and "data/50_ape.txt".
nombres = np.loadtxt("data/50_nombres.txt", dtype="str")
apellidos = np.loadtxt("data/50_ape.txt", dtype="str")

cuentas = []
for n in range(50):
        # We create a random name for the account holder.
        nombre = np.random.choice(nombres).title()
        ape1 = np.random.choice(apellidos).title()
        ape2 = np.random.choice(apellidos).title()
        titular = " ".join((nombre, ape1, ape2))
        # We create an account
        cuenta = Cuenta(titular, 2500)
        # Do 25 operations
        for n in range(25):
            cantidad = np.random.randint(200)
            tipo_op = np.random.choice(["R", "I"])
            if tipo_op == "R":
                cuenta.retirar(cantidad)
            else:
                cuenta.ingresar(cantidad)
        cuentas.append(cuenta)

# Save the list of accounts as an array in "data/accounts_allocations.npy" # Save the list of accounts as an array in "data/accounts_allocations.npy".
np.save("data/cuentas_aleatorias.npy", np.array(cuentas))

In [20]:
# Reads the accounts from the saved file
# Prints the owners of accounts 10 to 13 and their balances
cuentas = np.load("data/cuentas_aleatorias.npy")
for n in range(9, 13):
    print "Cuenta #{0}".format(n+1)
    print "Titular: " + cuentas[n]._nombre
    cuentas[n].consulta_saldo()
    print "_"*20

Cuenta #10
Titular: Fernando Diaz Sanz
Tu saldo en cuenta es de: 2577
____________________
Cuenta #11
Titular: Juan Rubio Martin
Tu saldo en cuenta es de: 3213
____________________
Cuenta #12
Titular: Alfonso Romero Martin
Tu saldo en cuenta es de: 2523
____________________
Cuenta #13
Titular: Ivan Alonso Marin
Tu saldo en cuenta es de: 3276
____________________


In [22]:
# Let's check the advantages of serialising objects with numpy versus normal serialisation.
import cPickle
import time

# Decorator function that will "time" how long it takes to execute a function
def cronometro(funcion):
    def wrapper(*args, **kwargs):
        # Tiempo de inicio de ejecución.
        inicio = time.time()
        # Ejecutamos la funcion
        ret = funcion(*args, **kwargs)
        # Tiempo de finalización de la ejecución.
        fin = time.time()
        print "La función se ha ejecutado en " + str(fin - inicio) + " segundos"
        return ret
    return wrapper

@cronometro
def guarda_objetos(lista_objetos, path):
    cPickle.dump(lista_objetos, open(path, "w"))

@cronometro
def guarda_objetos2(arr_objetos, path):
    np.save(path, arr_objetos)
    
@cronometro
def carga_objetos(path):
    return cPickle.load(open(path, "r"))

@cronometro
def carga_objetos2(path):
    return np.load(path)

nombres = np.loadtxt("data/50_nombres.txt", dtype = "|S15")
# Creamos una lista con 50000 cuentas con saldo inicial 1000 euros
lista_cuentas = [Cuenta(np.random.choice(nombres).title(), 1000) for n in range(50000)]
arr_cuentas = np.array(lista_cuentas)

guarda_objetos(lista_cuentas, "data/lista_cuentas.dat")
guarda_objetos2(arr_cuentas, "data/arr_cuentas.npy")

res = carga_objetos("data/lista_cuentas.dat")
res = carga_objetos2("data/arr_cuentas.npy")

La función se ha ejecutado en 1.16118097305 segundos
La función se ha ejecutado en 0.55837893486 segundos
La función se ha ejecutado en 0.89496588707 segundos
La función se ha ejecutado en 0.581037044525 segundos
